In [1]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
from scipy import fft
import os
import shutil

In [2]:
data_use = "train"
size_per_folder = 500


desired_sampling_rate = 16000 # in Hz
time_duration = 4 # in seconds

window_size = 108 #in ms
window_shift = 10 #in ms
pointFFT = 1728



source = "../Data/LA/"
destination = "../Data/main/"


window_size = int(window_size*desired_sampling_rate/1000) # converting to samples
window_shift = int(window_shift*desired_sampling_rate/1000) # converting to samples
labelfile = source + data_use + ".txt"

In [3]:
import warnings
warnings.filterwarnings("error")

bonafide_files = []
spoof_files = []
count = 0
count0 = 0
count1 = 0

with open(labelfile) as fp:
    for line in fp:
        contents = line.split(' ')
        audioname = contents[1] + '.flac'
        if contents[4] == 'bonafide\n': # bonafide
            bonafide_files.append(audioname) # 0 for bonaifde, aka not fake
            count0 += 1
        else: #spoofed
            spoof_files.append(audioname) # 1 for spoofed, aka fake
            count1 += 1
        count += 1
print("Total number of label entries found = ", count, len(bonafide_files)+len(spoof_files))
print("Total number of bonafide / non-fake samples found = ", count0)
print("Total number of spoofed / fake samples found = ", count1)

source_dir = source + data_use + "/flac/"

print("restructuring bonafide data...")

count = 0
for filename in bonafide_files:
    print(count%size_per_folder)
    if count%size_per_folder == 0:
        dest_dir = destination + data_use + "/bonafide_" + str(count//size_per_folder)
        os.mkdir(dest_dir)
    fname= os.path.join(source_dir,filename)
    print(fname)
    signal, sr = librosa.load(fname,sr = None)
    while np.shape(signal)[0] < time_duration*desired_sampling_rate:
        signal = np.hstack((signal,signal))
    signal = signal[:time_duration*desired_sampling_rate]
    try:
        spectrogram = np.log(np.abs(librosa.stft(signal, n_fft = window_size, hop_length = window_shift)))
    except RuntimeWarning:
        print(filename, "is excluded...")
        continue
    shutil.move(fname, dest_dir + "/" + filename)
    count += 1



print("restructuring spoof data...")
count = 0
for filename in spoof_files:
    print(count%size_per_folder)
    if count%size_per_folder == 0:
        dest_dir = destination + "/" + data_use + "/spoof_" + str(count//size_per_folder)
        os.mkdir(dest_dir)
    fname= os.path.join(source_dir,filename)
    signal, sr = librosa.load(fname,sr = None)
    while np.shape(signal)[0] < time_duration*desired_sampling_rate:
        signal = np.hstack((signal,signal))
    signal = signal[:time_duration*desired_sampling_rate]
    try:
        spectrogram = np.log(np.abs(librosa.stft(signal, n_fft = window_size, hop_length = window_shift)))
    except RuntimeWarning:
        print(filename, "is excluded...")
        continue
    shutil.move(fname, dest_dir + "/" + filename)
    count += 1

Total number of label entries found =  25380 25380
Total number of bonafide / non-fake samples found =  2580
Total number of spoofed / fake samples found =  22800
restructuring bonafide data...
0
../Data/LA/train/flac/LA_T_1138215.flac


DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
print(fname)